In [ ]:
import cPickle
import os
import collections
word_dict = collections.defaultdict(int)
for dirpath, dirnames, filenames in os.walk("data"):
    print("dirpath---%s---dirnames---%s---filenames---%s" % (str(dirpath), str(dirnames), str(filenames)))
    print("&"*100)
    if len(filenames) != 0:
        for fn in filenames:
            print("file name is %s" % fn)
            with open(os.path.join(dirpath, fn)) as f:
                for line in f:
                    print("which line is processing ---%s---" % line)
                    for w in line.strip().split():
                        print("which word is ---%s---" % w)
                        #print("which word is ---%s---" % w.decode('utf-8'))
                        word_dict[w] += 1
                        #word_dict[w.decode('utf-8')] += 1


In [ ]:
for key , value in word_dict.items():
    print("---key---%s---value---%d" % (key,value))

In [ ]:
items = list(word_dict.items())
items.sort(key=lambda x: x[1], reverse=True)
type(items)
for i in items:
    print("===%s======%s===" % (i[0],i[1]))

In [ ]:
print(items[0][0])

把词频给隐去了。。。。词频仅仅是为了作为排序的依据。。。所以。。len(iterms)有多长，数组就有多大。。。


In [ ]:
word_dict = dict()
for i in xrange(len(items)):
    word_dict[items[i][0]] = i
for key, value in word_dict.iteritems():
    print("---key---%s---value---%d" % (key,value))

In [ ]:
WORD_LIMIT =2000   # 只训练2000个词汇
WINDOW_SIZE=11    # 训练窗口大小为11
EMB_SIZE=32       # 设定词向量宽度
NUM_PASSES = 20   # 设定训练轮数

START_ID = WORD_LIMIT  # 句子开始标志
END_ID = START_ID + 1  # 句子结束标志
all_data = []
for dirpath, dirnames, filenames in os.walk("data"):
    for fn in filenames:
        print("文件名是：%s" %fn)
        with open(os.path.join(dirpath, fn)) as f:
            for line in f:
                print("没有加工之前---%s" % line)
                line = [word_dict[w] for w in line.strip().split() if word_dict[w] < WORD_LIMIT]
                print("加工完了之后---%s" % ' '.join(str(e) for e in line))
                line = [START_ID] + line + [END_ID]
                print("加上开始结束符---%s" % ' '.join(str(e) for e in line))
                if len(line) >= WINDOW_SIZE:
                    all_data.append(line)
                    

真他妈，牛逼。。。0-1999用来做词 ID。。。。2000，2001用来做句子的开始和结束。。。因为不可能重复。。。代码质量很高。。。。
下一步，开始配置 reader_creator...

In [ ]:
import random
def word_reader_creator():
    def reader():
        global all_data# 这个声明保证了all_data 可以被访问。。。
        random.shuffle(all_data)
        for line in all_data:
            print("$$$"*100)
            for i in xrange(len(line) - WINDOW_SIZE + 1):
                #从0开始，包含最后一个节点。。但是，list 是前闭后开。。。所以，正好进行
                print("start---%d---end%d---content---%s" % (i,i+WINDOW_SIZE, ' '.join(str(e) for e in line[i:i+WINDOW_SIZE])))
                yield line[i:i+WINDOW_SIZE]
    return reader

In [ ]:
for line in all_data:
    print("$$$"*100)
    for i in xrange(len(line) - WINDOW_SIZE + 1):
        #从0开始，包含最后一个节点。。但是，list 是前闭后开。。。所以，正好进行
        print("start---%d---end%d---content---%s" % (i,i+WINDOW_SIZE, ' '.join(str(e) for e in line[i:i+WINDOW_SIZE])))

这个代码实在是精妙。。。。11.。。前面是5个。。。后面是5个。。。。
正好把中间那个给剔除了。。。
第一：
非常重点来了。。。。全部转化为，，等长度的11.。。。前5个。。。后5个。。。。中间1个。。。正好11个。。。。
每个1层。。。。。。每层中的维度，最多2002种。。。。类型是词 ID。。。并不是真正的运算数据。。。
第二：
两个 for 循环。。。。
第二层把每个词 ID 最终用32个属性表示。。。。
并且把参数保存在名字叫 emb 的参数中。。。。
接下来对这个10个层，进行 map 操作，只不过是1对1的 map 操作，并没有发生什么变化。。。


In [ ]:
import paddle.v2 as paddle
paddle.init(use_gpu=False, trainer_count=3)
words = [paddle.layer.data(name="word_%d" % i, type=paddle.data_type.integer_value(WORD_LIMIT + 2)) for i in xrange(WINDOW_SIZE)]
embs = []
for w in words[:WINDOW_SIZE / 2 ] + words[-WINDOW_SIZE /2 + 1:]:
    embs.append(paddle.layer.embedding(input=w, size=EMB_SIZE, param_attr=paddle.attr.Param(name="emb", sparse_update=True)))
with paddle.layer.mixed(size=EMB_SIZE) as sum_emb:
    for emb in embs:
        sum_emb += paddle.layer.identity_projection(input=emb)
label = words[WINDOW_SIZE / 2]
cost = paddle.layer.hsigmoid(input=sum_emb, label = label, num_classes=WORD_LIMIT + 2)

In [ ]:
parameters = paddle.parameters.create(cost)
optimizer = paddle.optimizer.RMSProp(learning_rate=1e-3)
trainer = paddle.trainer.SGD(cost,parameters,optimizer)

In [ ]:
!mkdir -p output
import sys
import gzip
total_cost = 0.0
counter = 0
prefix="./output"
def event_handler(event):
    global total_cost
    global counter
    if isinstance(event, paddle.event.EndIteration):
        total_cost += event.cost
        count += 1
        sys.stdout.write('.')
        if event.batch_id % 100 == 0:
            print("Pass %d, Batch %d, AvgCost %f" %(event.pass_id, event.batch_id, total_cost/ counter))
        if event.batch_id % 10000 == 0:
            with gzip.open(os.path.join(prefix, "model_%d_%d.tar.gz" % (event.pass_id, event.batch_id)), 'w') as f:
                patameters.to_tar(f)

In [ ]:
trainer.train(
    paddle.batch(
    paddle.reader.buffered(word_reader_creator(), 16 * 4000), 
        3000),
    num_passes=NUM_PASSES, event_handler=event_handler, feeding=[w.name for w in words])